In [45]:
import pandas as pd
import numpy as np
import re
import time

# Load dataset
[Source Kaggle](https://www.kaggle.com/competitions/sentiment-analysis-on-movie-reviews/data)
### classes
* 0 - negative
* 1 - somewhat negative
* 2 - neutral
* 3 - somewhat positive
* 4 - positive

In [46]:
df = pd.read_csv("https://raw.githubusercontent.com/EnggQasim/Saylani-AI-Batch3/main/MachineLearning/NLP/train.tsv", 
                 sep="\t")
df.info()
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [47]:
df.Sentiment.value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [48]:
df1 = pd.concat([
    df[df.Sentiment==0], # negative class dataset
    df[df.Sentiment==4] # positive class dataset
])[["Phrase","Sentiment"]]

print(len(df1))

16278


In [49]:
df1.Sentiment = df1.Sentiment.map({0:0,
                                  4:1})#0=Negative 1=Postive remap values
df1

,Phrase,Sentiment
101,would have a hard time sitting through this one,0
103,have a hard time sitting through this one,0
157,Aggressive self-glorification and a manipulati...,0
159,self-glorification and a manipulative whitewash,0
201,Trouble Every Day is a plodding mess .,0
...,...,...
155946,is laughingly enjoyable,1
155955,a unique culture that is presented with univer...,1
155961,with universal appeal,1
156007,really do a great job of anchoring the charact...,1


In [50]:
df1.head()

,Phrase,Sentiment
101,would have a hard time sitting through this one,0
103,have a hard time sitting through this one,0
157,Aggressive self-glorification and a manipulati...,0
159,self-glorification and a manipulative whitewash,0
201,Trouble Every Day is a plodding mess .,0


## Import packages

In [51]:
import unidecode
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#pip install autocorrect
from autocorrect import Speller
import string
import timeit
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to /home/mqasim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Remove newlines & tabs

In [52]:
def remove_newlines_tabs(text):
    """
    This function will remove all the occurrences of newlines, tabs, and combinations like: \\n, \\.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of newlines, tabs, \\n, \\ characters.
        
    Example:
    Input : This is her \\ first day at this place.\n Please,\t Be nice to her.\\n
    Output : This is her first day at this place. Please, Be nice to her. 
    
    """
    
    # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
    Formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
    return Formatted_text

remove_newlines_tabs("This is her \\ first day at this place.\n Please,\t Be nice to her.\\n")

'This is her   first day at this place.  Please,  Be nice to her. '

In [53]:
print("This is her \\ first day at this place.\n Please,\t Be nice to her.\\n")

This is her \ first day at this place.
 Please,	 Be nice to her.\n


### Strip Html Tags


In [54]:
def strip_html_tags(text):
    """ 
    This function will remove all the occurrences of html tags from the text.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of html tags.
        
    Example:
    Input : This is a nice place to live. 
    Output : This is a nice place to live.  
    """
    # Initiating BeautifulSoup object soup.
    soup = BeautifulSoup(text, "html.parser")
    # Get all the text other than html tags.
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


strip_html_tags(" This is a <b>nice</b> place to live.<h1>Hello Wolrld</h1> ")

' This is a  nice  place to live. Hello Wolrld  '

In [55]:
print("This is a <b>nice</b> place to live.<h1>Hello Wolrld</h1>")

This is a <b>nice</b> place to live.<h1>Hello Wolrld</h1>


### Remove Links

In [56]:
def remove_links(text):
    """
    This function will remove all the occurrences of links.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of all types of links.
        
    Example:
    Input : To know more about cats and food & website: catster.com  visit: https://catster.com//how-to-feed-cats

    Output : To know more about cats and food & website: visit:     
    
    """
    
    # Removing all the occurrences of links that starts with https
    remove_https = re.sub(r'http\S+', '', text)
    # Remove all the occurrences of text that ends with .com
    remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
    return remove_com

remove_links(" website: catster.com  visit: https://catster.com//how-to-feed-cats adasfas ad as")

' website:   visit:  adasfas ad as'

In [57]:
print("website: catster.com  visit: https://catster.com//how-to-feed-cats adasfas ad as")

website: catster.com  visit: https://catster.com//how-to-feed-cats adasfas ad as


# Remove WhiteSpaces

In [58]:
def remove_whitespace(text):
    """ This function will remove 
        extra whitespaces from the text
    arguments:
        input_text: "text" of type    "String". 
                    
    return:
        value: "text" after extra whitespaces removed .
        
    Example:
    Input : How   are   you   doing   ?
    Output : How are you doing ?     
        
    """
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', text)
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    return text    

remove_whitespace("How   are  \t you \n   doing? (pakistan)ABC hello                world!")

'How are you doing ?  (pakistan) ABC hello world!'

In [59]:
"How   are  \t you \n   doing? (pakistan)ABC"

'How   are  \t you \n   doing? (pakistan)ABC'

### Remove Accented Characters

In [60]:
# Code for accented characters removal
def accented_characters_removal(text):
    # this is a docstring
    """
    The function will remove accented characters from the 
    text contained within the Dataset.
       
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" with removed accented characters.
        
    Example:
    Input : Málaga, àéêöhello
    Output : Malaga, aeeohello    
        
    """
    # Remove accented characters from text using unidecode.
    # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
    text = unidecode.unidecode(text)
    return text

accented_characters_removal("Málaga, àéêöhello")

'Malaga, aeeohello'

# Case Conversion

In [61]:
# Code for text lowercasing
def lower_casing_text(text):
    
    """
    The function will convert text into lower case.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
         value: text in lowercase
         
    Example:
    Input : The World is Full of Surprises!
    Output : the world is full of surprises!
    
    """
    # Convert text to lower case
    # lower() - It converts all upperase letter of given string to lowercase.
    text = text.lower()
    return text

lower_casing_text("Pakistan Zinda BAD!")

'pakistan zinda bad!'

### Reduce repeated characters and punctuations¶

In [62]:
def reducing_incorrect_character_repeatation(text):
    """
    This Function will reduce repeatition to two characters 
    for alphabets and to one character for punctuations.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Finally formatted text with alphabets repeating to 
        two characters & punctuations limited to one repeatition 
        
    Example:
    Input : Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)
    Output : Reallyy, Greeaatt !?.;:)
    
    """
    # Pattern matching for all case alphabets
    Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    
    # Limiting all the  repeatation to two characters.
    Formatted_text = Pattern_alpha.sub(r"\1\1", text) 
    
    # Pattern matching for all the punctuations that can occur
    Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
    
    # Limiting punctuations in previously formatted string to only one.
    Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)
    
    # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
    Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
    return Final_Formatted

reducing_incorrect_character_repeatation("Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)")

'Reallyy, Greeaatt !?.;:)'

# Expand contraction words

In [63]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
}
# The code for expanding contraction words
def expand_contractions(text, contraction_mapping =  CONTRACTION_MAP):
    """expand shortened words to the actual form.
       e.g. don't to do not
    
       arguments:
            input_text: "text" of type "String".
         
       return:
            value: Text with expanded form of shorthened words.
        
       Example: 
       Input : ain't, aren't, can't, cause, can't've
       Output :  is not, are not, cannot, because, cannot have 
    
     """
    # Tokenizing text into tokens.
    list_Of_tokens = text.split(' ')

    # Checking for whether the given token matches with the Key & replacing word with key's value.
    
    # Check whether Word is in lidt_Of_tokens or not.
    for Word in list_Of_tokens: 
        # Check whether found word is in dictionary "Contraction Map" or not as a key. 
         if Word in CONTRACTION_MAP: 
                # If Word is present in both dictionary & list_Of_tokens, replace that word with the key value.
                list_Of_tokens = [item.replace(Word, CONTRACTION_MAP[Word]) for item in list_Of_tokens]
                
    # Converting list of tokens to String.
    String_Of_tokens = ' '.join(str(e) for e in list_Of_tokens) 
    return String_Of_tokens    

expand_contractions("ain't , aren't , can't , cause , can't've")
   

"is not , are not , cannot , cause , cannot've"

# Remove special characters

In [64]:
# The code for removing special characters
def removing_special_characters(text):
    """Removing all the special characters except the one that is passed within 
       the regex to match, as they have imp meaning in the text provided.
   
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text with removed special characters that don't require.
        
    Example: 
    Input : Hello, K-a-j-a-l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) 
    Output :  Hello, Kajal. This is $100.05 : the payment that you will recieve! Is this okay?
    
   """
    # The formatted text after removing not necessary punctuations.
    Formatted_Text = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', text) 
    # In the above regex expression,I am providing necessary set of punctuations that are frequent in this particular dataset.
    return Formatted_Text

removing_special_characters(" Hello, K-a-j-a-l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) ")

' Hello, K a j a l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) '

### Remove stopwords

In [65]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/mqasim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [66]:
# The code for removing stopwords
stoplist = stopwords.words('english') 

stoplist = set(stoplist)
def removing_stopwords(text):
    """This function will remove stopwords which doesn't add much meaning to a sentence 
       & they can be remove safely without comprimising meaning of the sentence.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text after omitted all stopwords.
        
    Example: 
    Input : This is Kajal from delhi who came here to study.
    Output : ["'This", 'Kajal', 'delhi', 'came', 'study', '.', "'"] 
    
   """
    # repr() function actually gives the precise information about the string
    text = repr(text)
    # Text without stopwords
    No_StopWords = [word for word in word_tokenize(text) if word.lower() not in stoplist ]
    # Convert list of tokens_without_stopwords to String type.
    words_string = ' '.join(No_StopWords)    
    return words_string

removing_stopwords("This is Kajal from the delhi who was came here to study.")

"'This Kajal delhi came study . '"

# Correct mis-spelled words in text

In [67]:
# The code for spelling corrections
def spelling_correction(text):
    ''' 
    This function will correct spellings.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text after corrected spellings.
        
    Example: 
    Input : This is Oberois from Dlhi who came heree to studdy.
    Output : This is Oberoi from Delhi who came here to study.
      
    
    '''
    # Check for spellings in English language
    spell = Speller(lang='en')
    Corrected_text = spell(text)
    return Corrected_text

spelling_correction("This is Oberois from Dlhi who came hiree to stuydd.")

'This is Oberoi from Delhi who came hired to study.'

# Lemmatization

In [68]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/mqasim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/mqasim/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [69]:
# The code for lemmatization
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatization(text):
    """This function converts word to their root words 
       without explicitely cut down as done in stemming.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text having root words only, no tense form, no plural forms
        
    Example: 
    Input : text reduced 
    Output :  text reduce
    
   """
    # Converting words to their root forms
    lemma = [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]
    return lemma

lemmatization("cats ext having root words only, no tense form, no plural are forms cats dogs")

['cat',
 'ext',
 'have',
 'root',
 'word',
 'only,',
 'no',
 'tense',
 'form,',
 'no',
 'plural',
 'be',
 'form',
 'cat',
 'dog']

In [70]:
lemmatizer.lemmatize('educations','n')

'education'

# Putting all in single function

In [72]:
# Writing main function to merge all the preprocessing steps.
def text_preprocessing(text, accented_chars=True, contractions=True, lemmatization_allow = True,
                        extra_whitespace=True, newlines_tabs=True, repeatition=True, 
                       lowercase=True, punctuations=True, mis_spell=True,
                       remove_html=True, links=True,  special_chars=True,
                       stop_words=True):
    """
    This function will preprocess input text and return
    the clean text.
    """
    
    if newlines_tabs == True: #remove newlines & tabs.
        Data = remove_newlines_tabs(text)
        
    if remove_html == True: #remove html tags
        Data = strip_html_tags(Data)
        
    if links == True: #remove links
        Data = remove_links(Data)
        
    if extra_whitespace == True: #remove extra whitespaces
        Data = remove_whitespace(Data)
        
    if accented_chars == True: #remove accented characters
        Data = accented_characters_removal(Data)
        
    if lowercase == True: #convert all characters to lowercase
        Data = lower_casing_text(Data)
        
    if repeatition == True: #Reduce repeatitions   
        Data = reducing_incorrect_character_repeatation(Data)
        
    if contractions == True: #expand contractions
        Data = expand_contractions(Data)
    
    if punctuations == True: #remove punctuations
        Data = removing_special_characters(Data)
    
    stoplist = stopwords.words('english') 
    stoplist = set(stoplist)
    
    if stop_words == True: #Remove stopwords
        Data = removing_stopwords(Data)
        
    spell = Speller(lang='en')
    
    if mis_spell == True: #Check for mis-spelled words & correct them.
        Data = spelling_correction(Data)
        
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
#     print(Data)
    if lemmatization_allow == True: #Converts words to lemma form.
        Data = lemmatization(Data)
    
           
    return Data
text_preprocessing(' would have a hard time sitting through this one dogs ')

["'", 'would', 'hard', 'time', 'sit', 'one', 'dog', "'"]

# Now apply function on dataset

In [73]:
df1.head()

,Phrase,Sentiment
101,would have a hard time sitting through this one,0
103,have a hard time sitting through this one,0
157,Aggressive self-glorification and a manipulati...,0
159,self-glorification and a manipulative whitewash,0
201,Trouble Every Day is a plodding mess .,0


In [74]:
%%time
df1['input_data'] = df1.Phrase.apply(text_preprocessing)
df1.head()

/home/mqasim/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


CPU times: user 39min 54s, sys: 32.4 s, total: 40min 26s
Wall time: 40min 37s


,Phrase,Sentiment,input_data
101,would have a hard time sitting through this one,0,"['would, hard, time, sit, one, ']"
103,have a hard time sitting through this one,0,"['have, hard, time, sit, one, ']"
157,Aggressive self-glorification and a manipulati...,0,"['aggressive, self, clarification, manipulativ..."
159,self-glorification and a manipulative whitewash,0,"['self, clarification, manipulative, whitewash..."
201,Trouble Every Day is a plodding mess .,0,"['trouble, every, day, pad, mess, ., ']"


In [75]:
df1.Phrase = df1.Phrase.apply(str)
df1.head()

,Phrase,Sentiment,input_data
101,would have a hard time sitting through this one,0,"['would, hard, time, sit, one, ']"
103,have a hard time sitting through this one,0,"['have, hard, time, sit, one, ']"
157,Aggressive self-glorification and a manipulati...,0,"['aggressive, self, clarification, manipulativ..."
159,self-glorification and a manipulative whitewash,0,"['self, clarification, manipulative, whitewash..."
201,Trouble Every Day is a plodding mess .,0,"['trouble, every, day, pad, mess, ., ']"


In [77]:
df1.reset_index(drop=True).to_feather("data.feather")

In [78]:
df1.to_pickle("data.pkl")

In [79]:
df2 = pd.read_pickle("data.pkl")
df2.head()

,Phrase,Sentiment,input_data
101,would have a hard time sitting through this one,0,"['would, hard, time, sit, one, ']"
103,have a hard time sitting through this one,0,"['have, hard, time, sit, one, ']"
157,Aggressive self-glorification and a manipulati...,0,"['aggressive, self, clarification, manipulativ..."
159,self-glorification and a manipulative whitewash,0,"['self, clarification, manipulative, whitewash..."
201,Trouble Every Day is a plodding mess .,0,"['trouble, every, day, pad, mess, ., ']"


# https://www.analyticsvidhya.com/blog/2021/06/part-5-step-by-step-guide-to-master-nlp-text-vectorization-approaches/#:~:text=To%20convert%20the%20text%20data,text%20data%20to%20numerical%20vectors.

# https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
text = ["The quick brown fox jumped over the lazy dog."]
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_,"Vocab")
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape,"Shape")
print(type(vector),"Type")
print(vector.toarray(),"Array")

In [ ]:
text1 = ["the fox jumped"]
vectorizer.transform(text1).toarray()

 ## TF-IDF 
* Term Frequency: This summarizes how often a given word appears within a document.
* Inverse Document Frequency: This downscales words that appear a lot across documents.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents
text = ["The quick brown fox jumped over the lazy dog.",
 "The dog.",
 "The fox"]
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector = vectorizer.transform([text[0]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

In [ ]:
# Bag of Words
# n-gram
# TF-IDF
# Hashing with HashingVectorizer


# WordEmbeding